In [ ]:
# import all required libraries

import time
import numpy as np
import math
import scipy
from scipy.sparse import rand
import scipy.io as sio
from multiprocessing import Pool
from sklearn.linear_model import Ridge
from joblib import Parallel, delayed
from reservoirpy.observables import rmse
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import random
import concurrent.futures
import csv
from joblib import Parallel, delayed


# loading features and targets 

features = np.load('features_paper.npy',allow_pickle=True).T
n_features = int(features.shape[0])
actions = np.load('targets_paper.npy',allow_pickle=True).T

print(np.shape(actions))

#functions for using differnet combinations of hyper parameters
def Hyperparameters():

    betha = [0.001] #Input Gain
    alpha = [0.1] #Feedback Gain
    gamma = [0.5] #Interconnectivity Gain
    rho = [0.001] #Interconnectivity Matrix Density
    res_len = [32] 
    
    n_runs = len(betha)*len(alpha)*len(gamma)*len(rho)*len(res_len)
    scan_list =  product(betha,alpha,gamma,rho,res_len) 

    
    return scan_list,n_runs

def product(*args, repeat=1):

    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        tuple(prod)
    return result


# activation function 
def func(x):
    return np.sin(x)**2

# function for generating Win and W matrices
def matrices(beta,alpha,gamma,rho,n_features,s_reser):

    # input mask & input weights
    rng = np.random.default_rng(seed=42)
    mask0 = 2*rng.random((s_reser, n_features))-1
    rand_sgn  = 2*rng.integers(2, size=(s_reser,s_reser))-1
    mask = beta * mask0
    
    # reservoir weights
    wres = np.zeros((s_reser, s_reser))
    rng = np.random.default_rng(seed=43)
    wres = rand(s_reser, s_reser, density = rho, format='coo', dtype=None, random_state=43)
    wres = wres.toarray() * rand_sgn * gamma 
    # Diagonal of wres on gain_fdb
    np.fill_diagonal(wres, 1*alpha)
    
    print('shape of Input mask',np.shape(mask))
    print('shape of reservoir weights',np.shape(wres))
    
    return mask, wres

In [ ]:
# function for creating reservoir sequentially 
def reservoir(Win,Wres,n_inputs,features,s_reser,len_reservoir):
    
    reservoir_data = np.zeros((s_reser+1, n_inputs))
    print('reservoir of size', np.shape(reservoir_data), ' created')
    reservoir_data[s_reser]=1
    for i in range(n_inputs):
        if i == 0 :
            x = np.dot(Win,features[:,i])
        else:
            x = np.dot(Win,features[:,i])+np.dot(Wres,reservoir_data[0:s_reser,i-1])
        x = np.reshape(x, (len_reservoir, len_reservoir))
        sgn_in = np.sign(x)
        mxb_in = np.uint8(np.round(255*abs(x)))
        mxb_in[mxb_in>255] = 255
        mxb_in[mxb_in<-255] = -255

        img = func(mxb_in)
        img = img * sgn_in / 255
        
        img = img.reshape((len(img)*len(img),1))
        reservoir_data[0:s_reser,i] = img[:,0]
        

    return reservoir_data

In [ ]:

def reservoir(Win, Wres, n_inputs, features, s_reser, len_reservoir):
    def process_input(i):
        if i == 0:
            x = np.dot(Win, features[:, i])
        else:
            x = np.dot(Win, features[:, i]) + np.dot(Wres, reservoir_data[0:s_reser, i-1])
        x = np.reshape(x, (len_reservoir, len_reservoir))
        sgn_in = np.sign(x)
        mxb_in = np.uint8(np.round(255 * abs(x)))
        mxb_in[mxb_in > 255] = 255
        mxb_in[mxb_in < -255] = -255

        img = func(mxb_in)
        img = img * sgn_in / 255

        img = img.reshape((len(img) * len(img), 1))
        reservoir_data[0:s_reser, i] = img[:, 0]

        

    reservoir_data = np.zeros((s_reser + 1, n_inputs))
    print('reservoir of size', np.shape(reservoir_data), ' created')
    reservoir_data[s_reser] = 1
    

    Parallel(n_jobs=-1, prefer="threads")(delayed(process_input)(i) for i in range(n_inputs))

    return reservoir_data

In [ ]:
# function for creating reservoir in parallel 
def process_input(i, Win, Wres, features, s_reser, len_reservoir, reservoir_data):
    if i == 0:
        x = np.dot(Win, features[:, i])
    else:
        x = np.dot(Win, features[:, i]) + np.dot(Wres, reservoir_data[0:s_reser, i-1])
    x = np.reshape(x, (len_reservoir, len_reservoir))
    sgn_in = np.sign(x)
    mxb_in = np.uint8(np.round(255 * abs(x)))
    mxb_in[mxb_in > 255] = 255
    mxb_in[mxb_in < -255] = -255

    img = func(mxb_in)
    img = img * sgn_in / 255

    img = img.reshape((len(img) * len(img), 1))
    reservoir_data[0:s_reser, i] = img[:, 0]


def reservoir(Win, Wres, n_inputs, features, s_reser, len_reservoir):
    reservoir_data = np.zeros((s_reser + 1, n_inputs))
    print('reservoir of size', np.shape(reservoir_data), ' created')
    reservoir_data[s_reser] = 1

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_input, i, Win, Wres, features, s_reser, len_reservoir, reservoir_data) for i in range(n_inputs)]
        for future in concurrent.futures.as_completed(futures):
            pass

    return reservoir_data

In [ ]:
scan_list,n_runs = Hyperparameters()

#generating dictionary for storing corresponding hyper-parameters, time, and score of model
params = {"Reservoir Size": 0.0,"Input Gain": 0.0, "Feedback Gain": 0.0, "Interconnectivity Gain": 0.0,'Interconnectivity Matrix Density':0.0,'Time':0.0,'score':0.0}
score_list = []


# main code
for i in range(n_runs):
    print(i,'/',n_runs,'th run')
    print('reservoir started')
    ing= scan_list[i][0]
    fdb= scan_list[i][1]
    inter = scan_list[i][2]
    w_d= scan_list[i][3]

    n_inputs = int(features.shape[1])
    reservoir_len = scan_list[i][4]
    units = reservoir_len**2

    start_time = time.time()

    Win,Wres = matrices(ing,fdb,inter,w_d,n_features,units)
    reservoir_img = reservoir(Win, Wres, n_inputs, features, units, reservoir_len)

#     N = 39662
#     print('reservoir finished')
# #     X_train = reservoir_img[:, :N]
# #     Y_train = actions[:, :N]
# #     X_test = reservoir_img[:, N:]
# #     Y_test = actions[:, N:]
    
#     X_train = reservoir_img[:, :N]
#     Y_train = actions[:, :N]
#     X_test = reservoir_img[:, :N]
#     Y_test = actions[:, :N]


#     ridge = Ridge(alpha=0.1)

#     # Training the Ridge Regression model on the training data
#     reservoir_model= ridge.fit(X_train.T, Y_train.T)
    
#     end_time = time.time()
#     training_time = end_time-start_time
    
#     # Evaluating the model on the testing data
#     Y_pred = reservoir_model.predict(X_test.T)




#     print(np.shape(X_train))
#     print(np.shape(Y_train))
#     print(np.shape(X_test))
#     print(np.shape(Y_test))
    
#     predicted = np.argmax(Y_pred,axis=1)
#     test = np.argmax(Y_test.T,axis=1)
#     predicted_class = [np.argmax(Y_pred,axis=1) for y_p in predicted]
#     test_class = [np.argmax(Y_test.T,axis=1) for y_t in test]
#     score = (accuracy_score(np.concatenate(test_class, axis=0),np.concatenate(predicted_class, axis=0))*100)
#     updated = [("Reservoir Size",units),("Input Gain",ing), ("Feedback Gain", fdb), ("Interconnectivity Gain", inter),('Interconnectivity Matrix Density',w_d),('Time',training_time),('score',score)]
#     print(updated)
#     for key, value in updated:
#         params[key] = value
#     score_list.append(params.copy())

In [ ]:
my_list = score_list

# Open a CSV file for writing
with open('score_results_tunning_parallel_1024.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=my_list[0].keys())

    # Write the header row
    writer.writeheader()

    # Write each dictionary as a row
    for row in my_list:
        writer.writerow(row)

In [ ]:
# For plotting ConfusionMatrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(test,predicted)
cm_perc = (cm.astype('float') / cm.sum(axis=1)[:,np.newaxis])*100

act = [" Boxen ", " Handclapping ", " Hand Waving ", " Jogging ",
            " Running ", " Walking "]

# acc_prec = np.diag(cm_perc)
# print(acc_prec)


sns.heatmap(cm_perc, annot=True, cmap='plasma',xticklabels=act,yticklabels=act,linewidth =.5)
plt.title('Confusion Matrix for ReservoirSize:%s,'% (units))
plt.xlabel('Predicted')
plt.ylabel('Ture')